In [1]:
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [3]:
!pip install segmentation-models-3D

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 658.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.7/511.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow

In [4]:
import segmentation_models_3D as sm

import os
import sys
import numpy as np
import pandas as pd
import glob
import random
random.seed(0)

import pydicom
import nibabel as nib
import SimpleITK as sitk

import keras
import tensorflow as tf
from keras.optimizers import Adam, SGD     # tf.keras.optimizers.Adam

np.set_printoptions(threshold=sys.maxsize)

Segmentation Models: using `tf.keras` framework.


In [5]:
import tensorflow as tf

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15033286008062968342
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16131031040
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3962535868211356375
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
xla_global_id: 416903419
]


In [6]:
print(sys.version)
print("tf version: ", tf.__version__)
print("Keras version: ", keras.__version__)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
tf version:  2.9.1
Keras version:  2.9.0


In [7]:
def read_nifti(image_paths, mask_paths):
    
    if len(image_paths) != len(mask_paths):
        """ValueError: when user gives an invalid value to a function."""
        raise ValueError('The number of input files and segmentation masks are different.')
        
    if len(image_paths) == 0:
        raise ValueError('There is no image in the directory.')
        
    if len(mask_paths) == 0:
        raise ValueError('There is no mask in the directory.')
        
    num_images = len(image_paths)
    for index in range(num_images):
#         print(index)
        image_path = image_paths[index]
#         print(image_path)
        mask_path = mask_paths[index]
#         print(mask_path)
        
        if not os.path.isfile(image_path):
            raise ValueError(f'File {iamge_path} does not exist.')
            
        if not os.path.isfile(mask_path):
            raise ValueError(f'File {mask_path} does not exist.')
            
    input_images = nib.load(image_paths[0]) # load one volume to get the dimensions (x,y,z)
#     
    (nx, ny, nz) = input_images.shape[0:3]
    input_images = np.ndarray((num_images, nx, ny, nz, 1), dtype= np.float32) # Shape must be (#images, width, height, #slices, 1). Last 1 is needed.
    mask_images = np.ndarray((num_images, nx, ny, nz, 1), dtype= np.float32) 
    
    for index in range(0, num_images):
        input_image = nib.load(image_paths[index])
        mask_image = nib.load(mask_paths[index])
        
        input_images[index, :, :, :, 0]= input_image.get_fdata(dtype= np.float32)
        mask_images[index, :, :, :, 0] = mask_image.get_fdata(dtype= np.float32)
        
        return input_images, mask_images

In [8]:
data_dir= '/kaggle/input/cynch-seg-seg/'
print(os.listdir(data_dir))

img_dir= os.path.join(data_dir, 'images/')
mask_vessel= os.path.join(data_dir, 'human_corrected_with_vessel_removal/')
mask_novessel= os.path.join(data_dir, 'human_corrected_without_vessel_removal/')


img_files = sorted(glob.glob(os.path.join(img_dir, '*.nii')))
mask_vessel_files = sorted(glob.glob(os.path.join(mask_vessel, '*.nii')))
mask_novessel_files = sorted(glob.glob(os.path.join(mask_novessel, '*.nii')))


print(f'Total # of images: {len(img_files)}' )
print(f' Total # of masks, no vessel: {len(mask_novessel_files)}')
print(f'Total #  of masks, with vessel: {len(mask_vessel_files)}')

['images', 'human_corrected_without_vessel_removal', 'human_corrected_with_vessel_removal']
Total # of images: 111
 Total # of masks, no vessel: 111
Total #  of masks, with vessel: 111


In [9]:
img_filenames = [os.path.basename(x) for x in img_files]


# test ratio: 25%, validation ration: 15%, train ratio: 60%
num_test= int(np.ceil(0.02 * len(img_filenames)))
test_images = random.sample(img_filenames, num_test)


train_val_images = [x for x in img_filenames if x not in test_images]
num_val= int(np.ceil(0.11 * len(train_val_images)))
val_images = random.sample(train_val_images, num_val)

train_images = [x for x in train_val_images if x not in val_images]

print(f' # train: {len(train_images)} \n # validation: {len(val_images)} \n # test {len(test_images)}')
    

 # train: 96 
 # validation: 12 
 # test 3


In [10]:
#### check train, test, validation data leaks
# [x for x in test_images if x in val_images]
# [i for i, j in zip(test_images, val_images) if i == j]
### list(set(test_images).intersection(set(val_images)))
### list(set(train_images).intersection(set(val_images)))
### list(set(train_images).intersection(set(test_images)))

In [11]:
#########################################
train_img_path = []
train_mask_path = []

for filename in train_images:
    train_img_path.append(os.path.join(img_dir, filename))
    train_mask_path.append(os.path.join(mask_vessel, filename))
#########################################
validation_img_paths = []
validation_mask_paths = []

for filename in val_images:
    validation_img_paths.append(os.path.join(img_dir, filename))
    validation_mask_paths.append(os.path.join(mask_vessel, filename))

# # #########################################
test_img_path = []
test_mask_path = []

for filename in test_images:
    test_img_path.append(os.path.join(img_dir, filename))
    test_mask_path.append(os.path.join(mask_vessel, filename))
# # #########################################
(train_images, train_masks) = read_nifti(train_img_path, train_mask_path)
(validation_images, validation_masks) = read_nifti(validation_img_paths, validation_mask_paths)
(test_images, test_masks) = read_nifti(test_img_path, test_mask_path)


print(train_images.shape)
print(validation_images.shape)
print(test_images.shape)
# (96, 224, 224, 35, 1)
# (12, 224, 224, 35, 1)
# (3, 224, 224, 35, 1)

size_x = train_images.shape[1]
size_y = train_images.shape[2]
size_z = train_images.shape[3]

print(size_x, size_y, size_z)      #  224 224 35

# Shape must be (#images, width, height, #slices, 1). Last 1 is needed.

(96, 224, 224, 35, 1)
(12, 224, 224, 35, 1)
(3, 224, 224, 35, 1)
224 224 35


______ 
# Get the number of calsses

In [12]:
mask_obj= nib.load(mask_vessel_files[0])
mask_data= mask_obj.get_fdata()

In [13]:
np.unique(mask_data)

array([0.        , 1.00004578, 1.99995422, 3.        , 4.00004578,
       4.99995422, 6.        , 7.00004578, 7.99995422, 9.        ])

In [14]:
 a, cnts = np.unique(mask_data, return_counts=True)
high_freq, high_freq_element = cnts.max(), a[cnts.argmax()]

for elements, counts in zip(a, cnts):
    print(elements, ": ", counts)
print('_ _ _ _ _ ' * 5)    
# print(f'number of classes: {len(a)}')

num_classes = len(a)
num_classes

0.0 :  1696241
1.0000457768328488 :  1506
1.9999542224686593 :  4838
2.999999999301508 :  3445
4.000045776134357 :  2318
4.999954221770167 :  20466
5.999999998603016 :  5964
7.000045775435865 :  5008
7.999954221071675 :  11306
8.999999997904524 :  5068
_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ 


10

________________________________

In [15]:
# https://github.com/ZFTurbo/segmentation_models_3D/blob/master/tst_keras.py

# model1 = sm.Unet(
#     'resnet34', 
#     encoder_weights='imagenet'
# )

# # binary segmentation (this parameters are default when you call Unet('resnet34')
# model2 = sm.FPN(
#     'densenet121', 
#     classes=1, 
#     activation='sigmoid'
# )

# # multiclass segmentation with non overlapping class masks (your classes + background)
# model3 = sm.Linknet(
#     'resnet34', 
#     classes=3, 
#     activation='softmax'
# )

# # multiclass segmentation with independent overlapping/non-overlapping class masks
# model4 = sm.PSPNet(
#     'resnet34', 
#     classes=3,
#     activation='sigmoid'
# )

# # If you need to specify non-standard input shape
# model5 = sm.Unet(
#     'resnet50', 
#     input_shape=(96, 128, 128, 6), 
#     encoder_weights=None
# )

# # All possible backbones: 'resnet18, 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'seresnet18', 'seresnet34', 'seresnet50', 'seresnet101', 'seresnet152', 'seresnext50', 'seresnext101', 
# 'senet154', 'resnext50', 'resnext101', 'vgg16', 'vgg19', 'densenet121', 'densenet169', 'densenet201', 'inceptionresnetv2', 'inceptionv3', 'mobilenet', 'mobilenetv2', 'efficientnetb0', 
# 'efficientnetb1', 'efficientnetb2', 'efficientnetb3', 'efficientnetb4', 'efficientnetb5', 'efficientnetb6', 'efficientnetb7', 'efficientnetv2-b1', 'efficientnetv2-b2', 'efficientnetv2-b3', 
# 'efficientnetv2-s', 'efficientnetv2-m', 'efficientnetv2-l'

________________________

In [16]:
BACKBONE = 'resnet50'
encoder_weights = None   # 'imagenet'
activation = 'softmax'
n_classes = 10
epochs = 15
batch_size = 2
n_samples =  train_images.shape[0]
steps_per_epoch = n_samples / batch_size

# input_shape = train_images.shape[:-1]    # (64, 224, 224, 35)
input_shape = (64, 224, 224, 35)
lr = 1e-5     # 0.0001
optimizer = Adam(learning_rate=lr)
val_steps = 1

dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [17]:
model = sm.FPN(BACKBONE, classes = n_classes, input_shape = (96, 224, 224, 35),  encoder_weights = encoder_weights,  activation = activation)

model.compile(optimizer = optimizer, loss = total_loss, metrics = metrics)

In [18]:
model5 = sm.Unet(
    'resnet50', 
    input_shape=(96, 224, 224, 35), 
    encoder_weights=None
)
model5.compile(optimizer = optimizer, loss = total_loss, metrics = metrics)
history = model5.fit(train_images, train_masks, validation_data=(validation_images, validation_masks), epochs = epochs, batch_size = 1, verbose = 1)

2022-06-07 19:51:24.805406: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 674365440 exceeds 10% of free system memory.
2022-06-07 19:51:25.446207: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 674365440 exceeds 10% of free system memory.
2022-06-07 19:51:26.115237: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 674365440 exceeds 10% of free system memory.
2022-06-07 19:51:26.597830: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 674365440 exceeds 10% of free system memory.


Epoch 1/15


ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '


In [ ]:
# model3 = sm.Unet(
#     'vgg19',
#     input_shape=(96, 224, 224, 35), 
#     classes=10, 
#     activation='softmax'
# )

# history = model3.fit(train_images, train_masks, validation_data=(validation_images, validation_masks), epochs = epochs, batch_size = 1, verbose = 1)